# Tutorial 1
Question proposed by teacher and answers done by Chensheng LUO of CentraleSupélec

## Exercise 1.1

The aim is to evaluate the reliability of a heat exchanger. Through testing we get the following
numbers for all modes of failures:
- Mean number of failures per 106 hours: 96.93  

We assume that the global failure rate of the heat exchanger is constant.

5. Estimate the failure rate using intervals of length 50000/10 = 5000h.

In [ ]:
import numpy as np
from scipy.stats import expon
import matplotlib.pyplot as plt


lambda_value=96.93e-6
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

## Statistic-Empirical
r = expon.rvs(size=1000,scale=1/lambda_value)
number= int((max(r)-min(r))/5000)
ax.hist(r, bins=number,density=True, alpha=0.5, label='Empirical distribution')

## Theoretical distribution
xlim = ax.get_xlim()
vx = np.arange(0, xlim[1], 5000)
vy = expon.pdf(vx, scale=1/lambda_value)
ax.plot(vx, vy, 'r--', label='Theoretical distribution')

ax.grid(True)
ax.legend(loc='upper right')


In [ ]:
## Estimation of failure rate
lis=[0]*(number+1)
for i in r:
    lis[int(i/5000)] += 1

a = sum(lis)
for i in range(len(lis)):
    lis[i]=lis[i]/a

failure=[0]*(number)
for j in range(len(failure)):
    remained = 0
    for k in range(j,len(lis)):
        remained += lis[k]
    failure[j] = lis[j]/remained/5000

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.bar(range(len(failure)), failure,alpha=0.5,label='Emperical failure rate')
vx = np.full(len(failure),lambda_value)
ax.plot(range(len(failure)), vx, 'r--', label='Theoretical failure rate')
ax.grid(True)
ax.legend()

6. Proceed again to the estimation of the failure rate with a Weibull distribution

In [ ]:
from scipy.stats import weibull_max
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

## Statistic-Empirical
r = weibull_max.rvs(2,size=1000)
number=10
ax.hist(r, bins=number,density=True, alpha=0.5, label='Empirical distribution')

## Theoretical distribution
xlim = ax.get_xlim()
step = (xlim[1]-xlim[0])/number
vx = np.arange(xlim[0], xlim[1], step)
vy = weibull_max.pdf(vx,2)
ax.plot(vx, vy, 'r--', label='Theoretical distribution')

ax.grid(True)
ax.legend(loc='upper right')

In [ ]:
## Estimation of failure rate
lis=[0]*(number+1)
for i in r:
    lis[int((i-xlim[0])/step)] += 1

a = sum(lis)
for i in range(len(lis)):
    lis[i]=lis[i]/a

failure=[0]*(number)
for j in range(len(failure)):
    remained = 0
    for k in range(j,len(lis)):
        remained += lis[k]
    failure[j] = lis[j]/remained

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.bar(range(len(failure)), failure,alpha=0.5,label='Emperical failure rate')
ax.grid(True)
ax.legend()

## Exercise 2.2

Let us consider the system below. We assume that the components have exponential lives (and rates $\lambda_{1}$, $\lambda_{2}$, $\lambda_{3}$).The parallel association made of components 2 and 3 is necessary to the survival of the system.  
However, they are sharing load (in all rigour the Reliability Block Diagram is no longer a viable representation of the system). This means that if any one among components 2 and 3 fails, the remaining component will have to support a higher load.  
This translates in turn to a higher failure rate, namely:  
- With load sharing: if Comp 3 fails: $\lambda_{2}$ becomes $\lambda_{2}'=4 \lambda_{2}$  
- With load sharing: if Comp 2 fails: $\lambda_{3}$ becomes $\lambda_{3}'=4 \lambda_{3}$  

2. Calculate the value of the MTTF using $\lambda_{1}=0.01h^{-1}$, $\lambda_{2}=0.02h^{-1}$, $\lambda_{3}=0.05h^{-1}$

In [ ]:
from math import exp
from scipy.integrate import quad

lambda1=0.01
lambda2=0.02
lambda2p=4*lambda2
lambda3=0.05
lambda3p=4*lambda3

# The reliability function
def Rs(t):
    return exp(-lambda1*t)*(exp(-(lambda2+lambda3)*t)+lambda2/(lambda2+lambda3-lambda3p)*exp(-lambda3p*t)*(1-exp(-(lambda2+lambda3-lambda3p)*t))+lambda3/(lambda2+lambda3-lambda2p)*exp(-lambda2p*t)*(1-exp(-(lambda2+lambda3-lambda2p)*t)))

# Intergartion!
res, err = quad(Rs, 0, 1000)

print("The numerical result is {:f} (+-{:g})"
    .format(res, err))

3. Generate random trajectories representative of those histories/scenarios using
    - X = 3 for ‘All components are functioning’
    - X = 2 for ‘Component 2 has failed and 3 supports the full load’
    - X = 1 for ‘Component 3 has failed and 2 supports the full load’
    - X = 0 for ‘Component 1 has failed or both component 2 and 3 have failed’
Estimate the MTTF numerical and compare your results

In [ ]:
import random
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

t=[]

for k in range(1000):
    X = [3]
    
    for i in range(1,100):
        # Random passation
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        r3 = random.uniform(0, 1)
        if X[i-1] == 3:
            if r2 < lambda2 and r3>lambda3:
                X.append(2)
            elif r3 < lambda3 and r2>lambda2:
                X.append(1)
            elif r1< lambda1 or (r2<lambda2 and r3<lambda3):
                X.append(0)
                t.append(i)
            else:
                X.append(3)
        elif X[i-1] == 2:
            if r1< lambda1 or r3 < lambda3p:
                X.append(0)
                t.append(i)
            else:
                X.append(2)
        elif X[i-1] == 1:
            if r1< lambda1 or r2 < lambda2p:
                X.append(0)
                t.append(i)
            else:
                X.append(1)
        else:
            X.append(0)
            
    #Plot figure
    ax.plot(range(100), X)

ax.grid(True)

# Calculate MTTF
import statistics
print('MTTF is',statistics.mean(t))